In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
from tqdm.auto import tqdm
from Comparative_Analysis import Sequence_Analysis_Routines as sar
from Comparative_Analysis import Utilities as util
from Comparative_Analysis import Alignment as align
from Comparative_Analysis import Blast_Functions as blastfn
from Comparative_Analysis import ORF_Functions as orffn
from goatools import obo_parser
import random
import copy
from joblib import Parallel, delayed
import os
import wget
import shutil
import subprocess
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Align.Applications import MuscleCommandline
import re
import shutil
from ftplib import FTP
import Bio.UniProt.GOA as GOA

In [6]:
full_run = True
project_dir = 'D:/Project_Data/Project_8'
datasets_dir = project_dir + '/Datasets'
output_dir = project_dir + '/Output'
r_scape_output_loc = project_dir + '/Output'
wsl_output_loc = util.wslname(output_dir)
seq_dir = 'D:/Actinobacteria_Ref_Rep_Lev_Complete'
blast_dir = 'D:/BLAST/actinobacteria_ref_rep_comp'
blast_db_name = 'actinobacteria_ref_rep_comp'
num_cores = 16
core_numbers = list(range(1, num_cores+1))
reference_species_filename = 'GCF_000195955.2_ASM19595v2_genomic.gbff'
species_list = util.list_files(seq_dir)

##### Load data for orthologs, RFAM, etc

In [15]:
intergenic_regions = util.list_dirs(r_scape_output_loc)
temp_rows = []
for id in intergenic_regions:
    if os.path.isfile(r_scape_output_loc + '/' + id + '/summary_3.txt'):
        with open(r_scape_output_loc + '/' + id + '/summary_3.txt', 'r') as f:
                print(id)
                for l in f:
                    if (not ('#' in l)):
                            a = l.split()
                            if float(a[15]) < 0.0000001:
                                print(l)
                                species = a[0]
                                seq_from = int(a[7])
                                seq_to = int(a[8])
                                seq_strand = a[9]
                                significant = a[16]
                                start = min(seq_from,seq_to)
                                stop = max(seq_from,seq_to)


MTB000001_IG_TBONLY
NC_000962.3          -         rscape_2             -          cm        1      150    10962    11111      +    no    1 0.64   0.0  194.9   1.8e-43 !   -

MTB000002_IG_TBONLY
NC_000962.3          -         rscape_2             -          cm        1      368    11255    11684      +    no    1 0.66  15.4  284.1   3.2e-75 !   -

NC_000962.3          -         rscape_2             -          cm        1      368  1177300  1176976      -    no    1 0.66   8.9  225.8   3.6e-59 !   -

NC_000962.3          -         rscape_2             -          cm        1      368  4199062  4198723      -    no    1 0.67  13.3  219.6   1.8e-57 !   -

NC_000962.3          -         rscape_2             -          cm        1      368  2581930  2582245      +    no    1 0.65   5.5  204.9     2e-53 !   -

MTB000003_IG_TBONLY
NC_000962.3          -         rscape_2             -          cm        1      183    25729    25911      +    no    1 0.70   5.7  225.5   9.7e-47 !   -

MTB000008_

####   Print out all hits for particular region  